In [1]:
import os
import pandas as pd
import requests
import base64

#using azure
API_KEY = "2DWhd5NlWo6U8lpUBSWgyCuCL1bgNUAlQyv4sF2Oq6TsNxszq97NJQQJ99AKACHrzpqXJ3w3AAABACOGzNKu"
ENDPOINT = "https://rg-01-hippa-standalone-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"

def call_gpt(query):
    headers = {
        "Content-Type": "application/json",
        "api-key": API_KEY,
    }
    
    # Payload for the request
    payload = {
    "messages" : [
            {"role": "system", "content": "Answer the question based only on the following context, which can include text and tables: "},
            {"role": "user", "content": f"{query}"},
        ],
      "temperature": 0.7,
      "top_p": 0.95,
      "max_tokens": 800
    }
    
    
    # Send request
    try:
        response = requests.post(ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
    except requests.RequestException as e:
        raise SystemExit(f"Failed to make the request. Error: {e}")

    # Handle the response as needed (e.g., print or process)
    #print(response.json())
    #return response.json().choices[0].message.content
    return response.json()["choices"][0]["message"]["content"]







In [2]:
# read in full PE notes
import pandas as pd
Cat = pd.read_excel(r'tumor_size_all.xlsx',sheet_name='Sheet1',header=0)
Cat.head()

,Randomized Study Number,Size at Greatest Dimension (S) ***Be sure this is from the note and not the Mohs episode!,Physical Exam (PE),Assessment & Plan (A&P),Addendum (A)
0,36,0.8,"Physical exam:\nGeneral: Well developed, well ...",1. Chronic actinic damage in the setting of hi...,Addendum 7/11/2023\n \nThe pathology results w...
1,240,0.6,Physical exam:\nGeneral: Well-dressed; well-no...,Procedure Orders\nBiopsy [486623795] ordered b...,Addendum 7/11/2023\nThe pathology results were...
2,32,0.3,Physical exam:\nGeneral: Well-dressed; well-no...,Assessment & Plan: \n1. Brown papule on the le...,Addendum 7/13/2023\n \nThe pathology results w...
3,111,6.5,Physical exam:\nVital signs: There were no vit...,Assessment & Plan: \nBiopsy-proven basal cell ...,NaN
4,197,3.5,"Physical exam:\nGeneral: Well developed, well ...",Assessment & Plan: \n1. Chronic actinic sun d...,Addendum 7/5/2023\n \nThe pathology results we...


In [3]:
print(len(Cat))

294


In [10]:
# Reads only: Physical Exam
# Gets an array of tumor sizes for each patient and stores them in an array for the below code segment to sort through

#sampleSize = len(Cat) # Change number to increase or decrease number of patients read from file
sampleSize = 1

trueResults = [0 for i in range(sampleSize)]
trueResults_final = [0 for i in range(sampleSize)]

output = [0 for i in range(sampleSize)]
output_final = [0 for i in range(sampleSize)]
for i in range(0, min((sampleSize ), len(Cat))):
    results = Cat.at[i, 'Physical Exam (PE)']
    trueResults[i-1] = Cat.at[i, 'Size at Greatest Dimension (S) ***Be  sure this is from the note and not the Mohs episode!']
    que = "Question: what are the tumor sizes? answer in only an array of the numbers, followed by the units"
    prompt = f"{results} \n {que}"
    ans = call_gpt(prompt)
    print(ans)


[0.8 x 0.8, 0.4 x 0.5] cm


In [11]:
print(results)

Physical exam:
General: Well developed, well nourished. No acute distress.
Neuro: Alert and oriented to environment
Psych: pleasant affect
Total skin excluding undergarment areas, which includes the head/face, neck, left and right arms, chest, back, abdomen, left and right legs, digits and/or nails, was examined
-There is/are hyperkeratotic erythematous macule(s) with overlying adherent scale involving the right helix, right cheek, right alar crease, and left cheek
-There is a 0.8 x 0.8 cm pearly pink papule with arborizing fines telangiectasias on the right superior temple at site of prior surgery.
-There is a 0.4 x 0.5 cm pearly pink papule with arborizing fines telangiectasias on the right inferior temple at site of prior surgery.
-There is/are well-healed surgical scar(s) without nodularity, erythema, or telangiectasia at sites of previous skin cancer surgeries
-There are eroded erythematous papules and plaques in a dermatomal distribution on the right lower back/abdomen
-No other 

In [12]:
# Reads only: Physical Exam
# Gets an array of tumor sizes for each patient and stores them in an array for the below code segment to sort through

#sampleSize = len(Cat) # Change number to increase or decrease number of patients read from file
sampleSize = 1

trueResults = [0 for i in range(sampleSize)]
trueResults_final = [0 for i in range(sampleSize)]

output = [0 for i in range(sampleSize)]
output_final = [0 for i in range(sampleSize)]
for i in range(0, sampleSize):
    results = Cat.loc[i, ['Physical Exam (PE)', 'Assessment & Plan (A&P)', 'Addendum (A)']]
    trueResults[i-1] = Cat.at[i, 'Size at Greatest Dimension (S) ***Be  sure this is from the note and not the Mohs episode!']
    que = "Question: what are the tumor sizes? answer in only an array of the numbers, followed by the units"
    prompt = f"{results[0]+results[1]+results[2]} \n {que}"
    ans = call_gpt(prompt)
    print(ans)

0.8 x 0.8 cm, 0.4 x 0.5 cm


In [4]:
results = Cat.loc[47, ['Physical Exam (PE)', 'Assessment & Plan (A&P)', 'Addendum (A)']]

In [5]:
print(results[0]+results[1]+results[2])

Physical exam:
General: Well-dressed; well-nourished.
Psych: Pleasant affect.
Neuro: Alert and oriented to person.
Skin: Full skin, which includes the head/face, neck, left and right arms, chest, back, abdomen, left and right legs, genitalia and/or groin and/or buttocks, digits and/or nails, was examined
-Well-circumscribed (1.0cm x 1.3cm) nodule with overlying hemorrhagic crusting on the left pre-tibial leg
- There is a erythematous macule with overlying adherent scale on the right cheek
- There is a well-circumscribed waxy brown hyperkeratotic plaque on the left superior chest.
- There are no other skin lesions of concern in the areas examined.

Assessment & Plan: 
1. Neoplasm of uncertain behavior 
- Undiagnosed new problem with uncertain prognosis.
- A) Left pre-tibial leg; Differential diagnosis keratoacanthoma vs. Squamous cell carcinoma vs. Other 
- Photo obtained
- Biopsy today as below
 
2. Chronic actinic skin damage and actinic keratoses (x 1, on the right cheek),
- Chronic 

In [6]:
que = "Question: what are the tumor sizes? answer in only an array of the numbers, followed by the units"
prompt = f"{results[0]+results[1]+results[2]} \n {que}"
ans = call_gpt(prompt)
print(ans)

1.0, 1.3, cm


In [18]:
print(results)

Physical Exam (PE)         Physical exam:\nGeneral: Well developed, well ...
Assessment & Plan (A&P)    1. Chronic actinic damage in the setting of hi...
Addendum (A)               Addendum 7/11/2023\n \nThe pathology results w...
Name: 0, dtype: object


In [20]:
que = "Question: what are the tumor sizes? answer in only an array of the numbers, followed by the units"
prompt = f"{results[0]} \n {que}"
ans = call_gpt(prompt)
print(ans)

[0.8, 0.8, 0.4, 0.5] cm


In [21]:
que = "Question: what are the tumor sizes? answer in only an array of the numbers, followed by the units"
prompt = f"{results[1]} \n {que}"
ans = call_gpt(prompt)
print(ans)

The document does not provide specific measurements for the tumor sizes.


In [22]:
que = "Question: what are the tumor sizes? answer in only an array of the numbers, followed by the units"
prompt = f"{results[2]} \n {que}"
ans = call_gpt(prompt)
print(ans)

0.8 x 0.8 cm, 0.4 x 0.5 cm
